## HowTo

- Open instagram page (e.g. https://www.instagram.com/explore/tags/cats/),
- Save it to project dir,
- Refresh notebook
- Select saved HTML from select list
- Run all below cells
- Select images which should be preserved in dataset
- Copy JSON urls list

In [1]:
import glob
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import json, re
import os 
import urllib.request
from pprint import pprint

In [7]:
source = widgets.Dropdown(
    options=list(glob.glob('*.html')),
    description='Source:',
    disabled=False,
)
display(source)

Dropdown(description='Source:', options=('Hasztag #catlover na Instagramie • Zdjęcia i filmy.html', 'index.htm…

In [17]:
def extract_posts(text):
    d = re.findall('<script type="text/javascript">window._sharedData = (.*)</script>', text)
    assert len(d) == 1

    d = json.loads(d[0].strip(';'))['entry_data']['TagPage'][0]['graphql']['hashtag']
    results = {}

    for n in ['edge_hashtag_to_media', 'edge_hashtag_to_top_posts', 'edge_hashtag_to_content_advisory', 'edge_hashtag_to_related_tags', 'edge_hashtag_to_null_state']:
        for node in d[n]['edges']:
            thumbnails = node['node']['thumbnail_resources']
            for img in thumbnails:
                if img['config_height'] == img['config_width'] == 640:
                    results[node['node']['shortcode']] = img['src']
    return results

with open(source.value) as file:
    POSTS = extract_posts(file.read())

In [18]:
def download_images(posts):
    results = {}
    for k, url in posts.items():
        path = os.path.join('.cache', f"{k}.jpg")
        if not os.path.exists(path):
            with urllib.request.urlopen(url) as response, open(path, 'wb') as output:
                output.write(response.read())
        with open(path, 'rb') as file:
            results[(k,url)] = file.read()
    return results

IMAGES = download_images(POSTS)

In [19]:
COLS = 4

IMG_WIDTH = 200
IMG_HEIGHT = 200

rows = []
ITEMS = list(IMAGES.items())
CHECKBOXES = {}


def on_value_change(change):
    output.clear_output()
    with output:
        print(json.dumps({k1: k2 for (k1,k2), c in CHECKBOXES.items() if c.value}))
    

for i in range(0, len(ITEMS), COLS):
    row = ITEMS[i:i+COLS]
    cols = []
    for k, img in row:
        image = widgets.Image(
            value=img, width=IMG_WIDTH, height=IMG_HEIGHT
        )

        button = widgets.Checkbox(description='Include', indent=False, value=True)
        button.layout.width='auto'
        button.observe(on_value_change, names='value')
        CHECKBOXES[k] = button
        
        box = widgets.VBox([image, button])
        cols.append(box)

    # Create a horizontal layout box, grouping all the columns together
    rows.append(widgets.HBox(cols))
    
output = widgets.Output()

rows.append(output)

# Create a vertical layout box, grouping all the rows together
widgets.VBox(rows)